## Import libraries for env variables

In [4]:
import os
from dotenv import load_dotenv

## Import the env variables

In [5]:
load_dotenv()

AZURE_COGNITIVE_SEARCH_SERVICE_NAME = os.environ.get("AZURE_COGNITIVE_SEARCH_SERVICE_NAME")
AZURE_COGNITIVE_SEARCH_INDEX_NAME = os.environ.get("AZURE_COGNITIVE_SEARCH_INDEX_NAME")
AZURE_COGNITIVE_SEARCH_API_KEY = os.environ.get("AZURE_COGNITIVE_SEARCH_API_KEY")

OPENAI_API_TYPE = "azure"
OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_BASE = os.environ.get("OPENAI_API_BASE")
OPENAI_API_VERSION = "2023-03-15-preview"


In [3]:
AZURE_COGNITIVE_SEARCH_SERVICE_NAME

'qms'

In [4]:
AZURE_COGNITIVE_SEARCH_INDEX_NAME

'azureblob-cars'

In [5]:
AZURE_COGNITIVE_SEARCH_API_KEY

'Bnv9iA4mMRchH8jWlmMJKRy5cwVFbn3rCBMuynQrPcAzSeAoXQwU'

In [6]:
OPENAI_API_KEY

'9ac347d13e834f288a2076ff9c7b418a'

## Create the retriever that returns to us relevant documents

In [6]:
from langchain.retrievers import AzureCognitiveSearchRetriever

retriever = AzureCognitiveSearchRetriever(content_key = "content")

In [7]:
docs11 = retriever.get_relevant_documents("From what country is BMW")
docs11

[Document(page_content='\nBMW, short for Bayerische Motoren Werke AG, is a renowned German automobile manufacturer \n\ncelebrated for its commitment to luxury, performance, and engineering excellence. Founded in 1916, \n\nBMW has established itself as a symbol of automotive prestige and driving pleasure, captivating \n\nenthusiasts worldwide. \n\n \n\n**A Legacy of Innovation:** \n\nBMW\'s journey began in Munich, Germany, where it initially focused on manufacturing aircraft engines. \n\nAfter World War I, the company shifted its attention to motorcycles and eventually automobiles. BMW\'s \n\nearly automotive offerings, such as the Dixi 3/15 and the iconic 328, set the stage for the brand\'s \n\nreputation for quality and performance. \n\n \n\n**The Ultimate Driving Machine:** \n\nOne of BMW\'s most famous slogans is "The Ultimate Driving Machine," a phrase that captures the \n\nessence of the brand\'s vehicles. BMW has consistently prioritized driving dynamics, ensuring that their \n\

## Change the metadata for each document

In [8]:

for content in docs11:
    # all_content.append(content.page_content)
    content.metadata = {'source': 'azure'}


In [9]:
docs11[0].metadata

{'source': 'azure'}

In [10]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader


In [11]:
loader = TextLoader("text.txt")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

docs

[Document(page_content='The Volvo S90: Where Elegance Meets Innovation, The Volvo S90, a flagship luxury sedan from the Swedish automaker, is a testament to the brand\'s commitment to safety, performance, and Scandinavian design principles. Introduced in 2016, the S90 immediately made a strong impression in the luxury sedan segment and has continued to evolve with each subsequent model year. Design and Aesthetics: At first glance, the Volvo S90 captivates with its sleek and understated design. Clean lines, a bold grille adorned with the iconic Volvo emblem, and distinctive Thor\'s Hammer LED headlights create an unmistakable presence on the road. The exterior design is a fusion of timeless elegance and modern sophistication, setting it apart from more ostentatious luxury cars. Step inside, and you\'re greeted with an interior that embodies Scandinavian luxury. Premium materials such as Nappa leather, real wood trim, and brushed metal accents adorn the cabin. The minimalist design philo

## Import libraries for embeddings and store

In [12]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [15]:
embedding = OpenAIEmbeddings()

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
query = "From what country is BMW"


In [16]:
db2 = Chroma.from_documents(docs11, embedding_function, persist_directory="databse")
docs = db2.similarity_search(query)

In [17]:
db3 = Chroma(persist_directory="databse", embedding_function=embedding_function)
docs1 = db3.similarity_search(query)
docs1

[Document(page_content='\nBMW, short for Bayerische Motoren Werke AG, is a renowned German automobile manufacturer \n\ncelebrated for its commitment to luxury, performance, and engineering excellence. Founded in 1916, \n\nBMW has established itself as a symbol of automotive prestige and driving pleasure, captivating \n\nenthusiasts worldwide. \n\n \n\n**A Legacy of Innovation:** \n\nBMW\'s journey began in Munich, Germany, where it initially focused on manufacturing aircraft engines. \n\nAfter World War I, the company shifted its attention to motorcycles and eventually automobiles. BMW\'s \n\nearly automotive offerings, such as the Dixi 3/15 and the iconic 328, set the stage for the brand\'s \n\nreputation for quality and performance. \n\n \n\n**The Ultimate Driving Machine:** \n\nOne of BMW\'s most famous slogans is "The Ultimate Driving Machine," a phrase that captures the \n\nessence of the brand\'s vehicles. BMW has consistently prioritized driving dynamics, ensuring that their \n\

In [19]:
retriever = db3.as_retriever()
docs2 = retriever.get_relevant_documents("From what country is BMW")

len(docs2)
docs2

[Document(page_content='\nBMW, short for Bayerische Motoren Werke AG, is a renowned German automobile manufacturer \n\ncelebrated for its commitment to luxury, performance, and engineering excellence. Founded in 1916, \n\nBMW has established itself as a symbol of automotive prestige and driving pleasure, captivating \n\nenthusiasts worldwide. \n\n \n\n**A Legacy of Innovation:** \n\nBMW\'s journey began in Munich, Germany, where it initially focused on manufacturing aircraft engines. \n\nAfter World War I, the company shifted its attention to motorcycles and eventually automobiles. BMW\'s \n\nearly automotive offerings, such as the Dixi 3/15 and the iconic 328, set the stage for the brand\'s \n\nreputation for quality and performance. \n\n \n\n**The Ultimate Driving Machine:** \n\nOne of BMW\'s most famous slogans is "The Ultimate Driving Machine," a phrase that captures the \n\nessence of the brand\'s vehicles. BMW has consistently prioritized driving dynamics, ensuring that their \n\

## Import the model 3.5 turbo

In [20]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA

llm = AzureChatOpenAI(
    deployment_name = "gpt35-uif54579",
    model_name = "gpt-35-turbo",
    temperature = 0.7
)

In [21]:
llm

AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-35-turbo', temperature=0.7, model_kwargs={}, openai_api_key='9ac347d13e834f288a2076ff9c7b418a', openai_api_base='https://sbzdfopenai.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None, deployment_name='gpt35-uif54579', openai_api_type='azure', openai_api_version='2023-03-15-preview')

## Create a chain and use the query in the model

In [22]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever = retriever)

In [23]:
query1 = 'From what country is BMW'
llm_response = chain(query1)
llm_response

{'query': 'From what country is BMW', 'result': 'BMW is from Germany.'}

In [24]:
llm_response['result']

'BMW is from Germany.'